In [1]:
from pystac_client import Client
import geopandas
import json

In [2]:
stac_endpts = {'lpdaac':'https://cmr.earthdata.nasa.gov/stac/LPCLOUD',
               'ornldaac':'https://cmr.earthdata.nasa.gov/stac/ORNL_CLOUD',
               'usgs_ls':'https://landsatlook.usgs.gov/stac-server',
               'earthsearch_aws':'https://earth-search.aws.element84.com/v0'
              }

In [3]:
infile = geopandas.read_file('../data/ne_w_agfields.geojson')

In [4]:
roi = json.loads(infile.to_json())['features'][0]['geometry']
roi

{'type': 'Polygon',
 'coordinates': [[[-101.67271614074707, 41.04754380304359],
   [-101.65344715118408, 41.04754380304359],
   [-101.65344715118408, 41.06213891056728],
   [-101.67271614074707, 41.06213891056728],
   [-101.67271614074707, 41.04754380304359]]]}

In [5]:
date_range = "2021/2022"

---

In [6]:
lp_cat = Client.open(stac_endpts['lpdaac'])

In [7]:
list(lp_cat.get_collections())

[<CollectionClient id=ASTGTM.v003>,
 <CollectionClient id=HLSL30.v2.0>,
 <CollectionClient id=HLSS30.v2.0>]

In [8]:
for col in lp_cat.get_all_collections(): 
    print(f"{col.id}: {col.title}")

ASTGTM.v003: ASTER Global Digital Elevation Model V003
HLSL30.v2.0: HLS Landsat Operational Land Imager Surface Reflectance and TOA Brightness Daily Global 30m v2.0
HLSS30.v2.0: HLS Sentinel-2 Multi-spectral Instrument Surface Reflectance Daily Global 30m v2.0


In [9]:
lp_collections = ['HLSL30.v2.0', 'HLSS30.v2.0']

In [10]:
lp_search = lp_cat.search(
    collections=lp_collections,
    intersects=roi,
    datetime=date_range,
    limit=100
)

In [11]:
lp_search.matched()

460

---

In [12]:
ornl_cat = Client.open(stac_endpts['ornldaac'])

In [13]:
[f'{x.id}: {x.title}' for x in ornl_cat.get_all_collections() if 'Daymet' in x.title and 'Daily' in x.title]

['Daymet_Daily_V4_1840.v4.2: Daymet: Daily Surface Weather Data on a 1-km Grid for North America, Version 4']

In [14]:
ornl_collections = ['Daymet_Daily_V4_1840.v4.2']

In [15]:
search = ornl_cat.search(
    collections=ornl_collections,
    intersects=roi,
    datetime=date_range,
    limit=100
)

In [16]:
search.matched()

7

In [33]:
search.get_all_items_as_dict()['features'][0]

{'type': 'Feature',
 'id': 'Daymet_Daily_V4.daymet_v4_daily_na_tmax_2021.nc',
 'stac_version': '1.0.0',
 'stac_extensions': [],
 'collection': 'Daymet_Daily_V4_1840.v4.2',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-178.133, 14.0749],
    [-53.0567, 14.0749],
    [-53.0567, 82.9143],
    [-178.133, 82.9143],
    [-178.133, 14.0749]]]},
 'bbox': [-178.133, 14.0749, -53.0567, 82.9143],
 'links': [{'rel': 'self',
   'href': 'https://cmr.earthdata.nasa.gov/stac/ORNL_CLOUD/collections/Daymet_Daily_V4_1840.v4.2/items/Daymet_Daily_V4.daymet_v4_daily_na_tmax_2021.nc'},
  {'rel': 'parent',
   'href': 'https://cmr.earthdata.nasa.gov/stac/ORNL_CLOUD/collections/Daymet_Daily_V4_1840.v4.2'},
  {'rel': 'collection',
   'href': 'https://cmr.earthdata.nasa.gov/stac/ORNL_CLOUD/collections/Daymet_Daily_V4_1840.v4.2'},
  {'rel': 'root', 'href': 'https://cmr.earthdata.nasa.gov/stac/'},
  {'rel': 'provider',
   'href': 'https://cmr.earthdata.nasa.gov/stac/ORNL_CLOUD'},
  {'rel': 'via',
   'href':

---

In [27]:
usgs_cat = Client.open(stac_endpts['usgs_ls'])

In [28]:
for col in usgs_cat.get_all_collections(): 
    print(f"{col.id}: {col.title}")

landsat-c1l2alb-st: Landsat Collection 1 Level-2 Albers Surface Temperature (ST) Product
landsat-c2l2-sr: Landsat Collection 2 Level-2 UTM Surface Reflectance (SR) Product
landsat-c2l2-st: Landsat Collection 2 Level-2 UTM Surface Temperature (ST) Product
landsat-c2ard-st: Landsat Collection 2 Analysis Ready Data (ARD) Level-2 UTM Surface Temperature (ST) Product
landsat-c2l2alb-bt: Landsat Collection 2 Level-2 Albers Top of Atmosphere Brightness Temperature (BT) Product
landsat-c2ard-bt: Landsat Collection 2 Analysis Ready Data (ARD) Level-2 UTM Top of Atmosphere Brightness Temperature (BT) Product
landsat-c2l3-fsca: Landsat Collection 2 Level-3 Fractional Snow Covered Area (fSCA) Product
landsat-c1l2alb-bt: Landsat Collection 1 Level-2 Albers Top of Atmosphere Brightness Temperature (BT) Product
landsat-c2l1: Landsat Collection 2 Level-1 Product
landsat-c2l3-ba: Landsat Collection 2 Level-3 Burned Area (BA) Product
landsat-c1l2alb-ta: Landsat Collection 1 Level-2 Albers Top of Atmosph

In [29]:
usgs_collections = ['landsat-c2l2-sr']

In [30]:
usgs_search = usgs_cat.search(
    collections=usgs_collections,
    intersects=roi,
    datetime=date_range,
    limit=100
)

In [31]:
usgs_search.matched()

200

In [34]:
usgs_search.get_all_items_as_dict()['features'][0]

{'type': 'Feature',
 'stac_version': '1.0.0',
 'stac_extensions': ['https://landsat.usgs.gov/stac/landsat-extension/v1.1.1/schema.json',
  'https://stac-extensions.github.io/view/v1.0.0/schema.json',
  'https://stac-extensions.github.io/projection/v1.0.0/schema.json',
  'https://stac-extensions.github.io/eo/v1.0.0/schema.json',
  'https://stac-extensions.github.io/alternate-assets/v1.1.0/schema.json',
  'https://stac-extensions.github.io/storage/v1.0.0/schema.json'],
 'id': 'LC09_L2SP_031032_20220524_20220526_02_T2_SR',
 'description': 'Landsat Collection 2 Level-2 Surface Reflectance Product',
 'bbox': [-102.52017264317804,
  39.269082961880436,
  -99.83776685928268,
  41.38035904580066],
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-102.00324954307321, 41.38035904580066],
    [-102.52017264317804, 39.66376682231712],
    [-100.4099067889537, 39.269082961880436],
    [-99.83776685928268, 40.98387531732431],
    [-102.00324954307321, 41.38035904580066]]]},
 'properties': {'date